In [ ]:
from splashback.cluster import cluster_sample
from astropy.table import Table, Column
from matplotlib import pyplot as plt 
import numpy as np 
from astropy.cosmology import FlatLambdaCDM
from astropy import units as u
from splashback.profile import Mvir_to_M200m, rvir

***Reformat catalogs***

The original source catalogs are in txt files, which can be difficult to parse. This files rewrites everything in FITS file using the same notation as in the paper. 

Notice that the contamination correction parameters are loaded from the original file as a correction on the sky-coordinates; this way the whole correction becomes cosmology-independent. 

In [ ]:
data = Table.read('data/original/clusters.dat', format='ascii')
data_planck = Table.read('data/original/m500_mega_planck.dat', format='ascii')

z, xcen, ycen, mmin, mmax, da, beta_avg, r_500,  m_500, n_0, r_core, r_max = [np.zeros(len(data)) for i in xrange(12)]
column_list = ['name', 'z', 'xcen', 'ycen', 'mmin', 'mmax', 'da', 'beta_avg', 'r_500',  'm_500', 'n_0', 'r_core', 'r_max']
source_column_list = ['x', 'y', 'm', 'e1', 'e2', 'de', 'pg', 'mu', 'delmag', 'e1r', 'e2r']
meta_var = {'details' : 'CCCP sample - Hoekstra+ 2015, Planck r_500, m_500'}
name = [None]*len(data)
da, r_500, m_500, n_0, r_core, r_max = da*u.Gpc/u.rad, r_500*u.arcsec, m_500*u.Msun, n_0*u.arcsec, r_core*u.arcsec, r_max*u.arcsec

i=0
for cluster_name in data['name_cl']:
    print cluster_name
    Mpc = data['Mpc'][data['name_cl']==cluster_name]

    name[i] = cluster_name
    z[i] = data['z_cl'][data['name_cl']==cluster_name]
    xcen[i] = data['xcen'][data['name_cl']==cluster_name]
    ycen[i] = data['ycen'][data['name_cl']==cluster_name]
    mmin[i] = data['mmin'][data['name_cl']==cluster_name]
    mmax[i] = data['mmax'][data['name_cl']==cluster_name]
    da[i] = data['da'][data['name_cl']==cluster_name]*u.Gpc/u.rad
    beta_avg[i] = data['beta_avg'][data['name_cl']==cluster_name]
    r_500[i] = data_planck['r_d'][data_planck['name_clus']==cluster_name]*Mpc*u.arcsec
    m_500[i] = data_planck['m_d'][data_planck['name_clus']==cluster_name]/0.7*u.Msun

    data_contam = np.loadtxt('data/original/CONTAM_PAR/'+cluster_name+'.par')
    n_0[i] = data_contam[6,4]/data_contam[6, 8]*u.arcsec
    r_core[i] = data_contam[6,6]*Mpc*0.7*u.arcsec
    r_max[i] = 4*Mpc*0.7*u.arcsec
    i+=1


    # LOAD SOURCE CATALOG
    data_source = np.loadtxt('data/original/source_new/'+cluster_name+'.cat')
    data_source_extra = np.loadtxt('data/original/source_old/mos_'+cluster_name+'.cat')

    source_meta_var = {'NAME' : cluster_name, 'SAMPLE' : 'CCCP', 'PIXSIZE' : 0.186, 'PIXUNIT' : 'arcsec'}
    Table_source = Table([data_source[:, 0], data_source[:, 1], data_source[:,2], data_source[:, 3], data_source[:,4], \
                        data_source[:, 5], data_source[:,6], data_source[:, 7], data_source[:,11], \
                        data_source_extra[:, 3], data_source_extra[:, 4]], names=source_column_list, meta=source_meta_var)
    Table_source.write('data/source/'+cluster_name+'.fits', overwrite=True)


#SAVE SAMPLE FITS TABLE
cluster_table = Table([name, z, xcen, ycen, mmin, mmax, da, beta_avg, r_500,  m_500, n_0, r_core, r_max], names=column_list, meta=meta_var)
cluster_table.write('data/CCCPoriginal.fits', overwrite=True)


***Update cosmology variables***

In [ ]:
filepath = "data/CCCPoriginal.fits" # CCCP original file 
N_bins = 6 # bins in magnitude


#---------------------------------------------------------------------

#Hardcoded
dirpath = "data/source/" # source file
photozpath = "data/photoz/COSMOS2015.fits" # redshift dist file
noisepath = "output/noise/" # noise dir where to save the weighted redshift dist. 

#Initialize things
CCCP = cluster_sample(filepath, dirpath)
data_z = Table.read(photozpath)
zlist = data_z['z'].quantity.value
mlist = data_z['m'].quantity.value
mbins = np.linspace(22, 25, N_bins+1)
beta = np.zeros(len(CCCP))
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

# Average beta for a z_s_list (list of source redshifts) and z_l (lens redshift)
def avg_beta(z_s_list, z_l):
    temp = cosmo.angular_diameter_distance_z1z2(z_l, z_s_list)/cosmo.angular_diameter_distance(z_s_list)
    temp[temp<0] = 0.
    return temp.mean()

# Compute beta for each cluster using magnitude weights
for i in xrange(len(CCCP)):
    w_avg = np.zeros(N_bins)
    beta_bin = np.zeros(N_bins)
    self = CCCP[i]
    z_l = CCCP[i].z.value
    idx = (self.m > self.mmin) & (self.m < self.mmax) & (self.pg>0.1) & (self.delmag == 0)
    x, y, pg, de, m, mu = self.x[idx], self.y[idx], self.pg[idx], self.de[idx], self.m[idx], self.mu[idx]
    w = pg**2./((0.25*pg)**2. + de**2.)
    data_z_temp = Table.read(photozpath)
    w_column = np.ones(len(data_z_temp))    
    
    for j in xrange(N_bins):
        # average weight in bin
        idx = (m >= mbins[j]) & (m < mbins[j+1])
        w_avg[j] = w[idx].sum()/w.sum()
        
        
        # average beta in bin
        idx = (mlist>=mbins[j]) & (mlist<mbins[j+1])
        beta_bin[j] = avg_beta(zlist[idx], z_l)    
        w_column[idx] = w_avg[j]/idx.sum()
    
    beta[i] = (beta_bin*w_avg).sum()

    data_z_temp.add_column(Column(w_column, name='w'))
    data_z_temp.write(noisepath+CCCP[i].name+".fits", overwrite=True)

***Divide subsamples***

In [ ]:
data_CCCP = Table.read(filepath)


data_CCCP['da'] = cosmo.angular_diameter_distance(data_CCCP['z'])/u.rad

m_g = np.array([0.60, 0.80, 0., 0.68, 0.85, 1.06, 0.65, 0.66, 1.56, 0.97, 0.75, 0., 0., 1.21, 0.99, 0.44, 0.68, 0.74, 2.33, 1.16, 0.50, 1.46, 0.86, 0.24, 1.03, 0.58, 1.63, 0.41, 2.35, 0.53, 0., 0.62, 0.61, 0.])*(1e14*u.Msun)
m_vir = np.array([5.9, 7.0, 8.9, 7.8, 15.3, 10.7, 4.6, 9.4, 14.1, 21.4, 19.7, 8.7, 19.4, 19.9, 13.5, 13.5, 15.7, 9.4, 17.4, 19.9, 7.9, 24.4, 20.9, 3.5, 17.4, 16.3, 19.9, 6.2, 38.4, 6.5, 17.2, 12.0, 6.4, 7.6])*(1e14*u.Msun)
off = np.array([11, 9, -1, 10, 0, 47, 14, 12, 20, 0, 39, -1, -1, 12, 0, 0.7, 10, 108, 11, 4, 75, 5, 8, 4, 34, 12, 7, 44, 113, 1, -1, 5, 14, -1])*u.kpc
m_200 = np.zeros(len(m_vir))*u.Msun

for i in xrange(len(m_vir)):
    m_200[i] = Mvir_to_M200m(m_vir[i], data_CCCP['z'][i])

idx = (m_g == 0.)
z = data_CCCP['z']
m_500 = data_CCCP['m_500'].quantity
Ez = np.sqrt(0.3*(1.+z.quantity)**3. +0.7)

m_g[idx] = ( (m_500*Ez)[idx]/(1e14*u.Msun) /(10**0.9) )**(1./1.04) /Ez[idx] * (1e14*u.Msun)

print "Special Mgas:"
print data_CCCP['name'][idx]

data_CCCP.add_column(Column(m_g, name='m_g'))
data_CCCP.add_column(Column(beta, name='beta'))
data_CCCP.add_column(Column(m_200, name='m_200'))
data_CCCP.add_column(Column(m_vir, name='m_vir'))
data_CCCP.add_column(Column(off, name='off'))

data_CCCP = data_CCCP['name', 'xcen', 'ycen', 'mmin', 'mmax', 'n_0', 'r_core', 'r_max', 'r_500', 'm_500', 'm_200', 'm_vir', 'm_g', 'z', 'da', 'beta', 'off']
data_CCCP.write('data/CCCP.fits', overwrite=True)

data_CCCP = Table.read('data/CCCP.fits')
idx = (data_CCCP['name'] != "A115S") & (data_CCCP['name'] != "A115N") & (data_CCCP['name'] != "A223S") &  (data_CCCP['name'] != "A223N") & (data_CCCP['name'] != "MACS0717") & (data_CCCP['name'] != "A222") & (data_CCCP['name'] != "A1758") 
data_CCCP = data_CCCP[idx]
data_CCCP.write('data/CCCPnomergers.fits', overwrite=True)

data_CCCP = Table.read('data/CCCPnomergers.fits')
idx = (data_CCCP['z'] > 0.25)
data_CCCP = data_CCCP[idx]
data_CCCP.write('data/CCCPhighz.fits', overwrite=True)

data_CCCP = Table.read('data/CCCPnomergers.fits')
idx = data_CCCP['m_g'].quantity > 8e13*u.Msun
data_CCCP = data_CCCP[idx]
data_CCCP.write('data/CCCPhighm.fits', overwrite=True)



***Prepare files for intrinsic covariance matrix***

In [ ]:
np.set_printoptions(suppress=True)
bin_edges = np.geomspace(.2, 9, 11)*u.Mpc
z_bin_edges = np.around(np.linspace(0.005, 5.995, 600), 5)

#------------------------------------------
data_CCCP = Table.read('data/CCCPnomergers.fits')
filep = open("output/noise/CCCP.tab", "w+")
for i in xrange(len(data_CCCP)):
    print data_CCCP['name'][i]
    # Write entry in main file
    filep.write(data_CCCP['name'][i]+" "+str(np.around(data_CCCP['z'][i], 2))+" "+data_CCCP['name'][i]+"_pz annuli"+data_CCCP['name'][i]+"\n")
    
    # Generate annuli file
    file_annuli = open("output/noise/annuli"+data_CCCP['name'][i]+".tab", "w+")
    bin_edges_sky = ( bin_edges/data_CCCP['da'].quantity[i] ).to('arcmin').value
    file_annuli.write(str(len(bin_edges_sky)-1)+"\n")
    for j in xrange(len(bin_edges_sky)-1):
        file_annuli.write(str(np.around(bin_edges_sky[j], 5))+" "+str( np.around(bin_edges_sky[j+1], 5))+"\n")
    file_annuli.close()
    
    # Generate photometric redshift file
    file_pz = open("output/noise/"+data_CCCP['name'][i]+"_pz.tab", "w+")
    file_pz.write("#hist\n")
    #file_pz.write(str(np.around(z_bin_edges[0], 5))+" "+str(np.around(z_bin_edges[-1], 5))+"\n")
    data_z = Table.read("data/photoz/individual/"+data_CCCP['name'][i]+".fits")
    zlist = data_z['z']
    wlist = data_z['w']
    for j in xrange(len(z_bin_edges)-1):
        idx = (zlist < z_bin_edges[j+1]) & (zlist >= z_bin_edges[j])
        value = np.around(wlist[idx].sum(), 15)
        file_pz.write(str(z_bin_edges[j])+" "+format(value, '.15f')+"\n")
    file_pz.write(str(z_bin_edges[-1])+" 0\n")
    file_pz.close()
filep.close()
